In [ ]:
import azureml.core
from azureml.core import Workspace
print(azureml.core.VERSION)

In [ ]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

In [ ]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)

In [ ]:
# notebook env value
subscription_id = '<subscription id>'
resource_group = '<resource group>'
workspace_name = 'dw-ws'

# create aml workspace or create it azure portal
#https://docs.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py#workspace

from azureml.core import Workspace
ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      create_resource_group=True,
                      location='eastus2'
                     )

In [ ]:
# not using private ACR, using dockerhub public
# for provate registry : https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-custom-docker-image
"""
acr_server = "<ACRNAME>.azurecr.io"
acr_login = "<ACRNAME>"
acr_pwd = "<PWD>"
acr_repo_name = "<REPO_NAME>"
"""

In [ ]:
from azureml.core import Workspace

# setup workspace - reloading
ws = Workspace.from_config()
ws.get_details()

In [ ]:
# setup datastore
ds = ws.get_default_datastore()
ds.name

In [ ]:
# upload data folder to ds
ds.upload(
    src_dir='./upload_data',
    target_path='tfdata',
    overwrite=True)

In [ ]:
"""
# train remote VM - gpu
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name='dw-gpu')
    print('found existing:', compute_target.name)
except ComputeTargetException:
    print('creating new.')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_NC6',
        min_nodes=0,
        max_nodes=1)
    compute_target = ComputeTarget.create(ws, 'dw-gpu', compute_config)
    compute_target.wait_for_completion(show_output=True)
"""

In [ ]:
# train remote VM - cpu
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name='dw-cpu')
    print('found existing:', compute_target.name)
except ComputeTargetException:
    print('creating new.')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_D3_V2',
        min_nodes=0,
        max_nodes=1)
    compute_target = ComputeTarget.create(ws, 'dw-cpu', compute_config)
    compute_target.wait_for_completion(show_output=True)


In [ ]:
ds.container_name

In [ ]:
# data reference
from azureml.core import Datastore
from azureml.core.runconfig import DataReferenceConfiguration

ds = Datastore.get(ws, datastore_name="workspaceblobstore")

dr_conf = DataReferenceConfiguration(
    datastore_name=ds.name,
    path_on_datastore='tfdata',
    #path_on_compute = '/tfdata'
    mode='mount') # or 'download'

In [ ]:
# check mount variable
str(ds.as_mount())

In [ ]:
# use estimator
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training-with-deep-learning/distributed-cntk-with-custom-docker/distributed-cntk-with-custom-docker.ipynb
from azureml.train.estimator import Estimator

params= {'--data_folder' : ds.as_mount() }

estimator = Estimator(source_directory='script',
                      compute_target=compute_target,
                      entry_script='train.py',
                      script_params=params,
                      node_count=1,
                      process_count_per_node=1,
                      pip_requirements_file_path = '../docker/requirements.txt', # pip packages
                      custom_docker_image='konan94/dw-tf-od:v8', # using public docker hub
                      #custom_docker_image='konan94/dw-tf-od-gpu:v9', 
                      use_gpu=False)

In [ ]:
from azureml.core import Experiment

experiment_name = 'dw-exp'
experiment = Experiment(ws, name=experiment_name)

run = experiment.submit(estimator)
print(run)
run.wait_for_completion(show_output=True)

In [ ]:
# delete cpu compute
"""
mycompute = AmlCompute(workspace=ws, name='dw-cpu')
mycompute.delete()

# delete gpu compute
mycompute = AmlCompute(workspace=ws, name='dw-gpu')
mycompute.delete()
"""

In [ ]:
# delete workspace
ws.delete(delete_dependent_resources=True)
